In [1]:
from keras.layers import LSTM, Dense, Reshape, Conv2D, MaxPooling2D, Flatten, Bidirectional
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [3]:
pos = pd.read_table('dataset/training.txt', names = ['y', 'x'])
pos = shuffle(pos)
data =pos['x']
y = pos['y']

In [4]:
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding, Input, TimeDistributed, Conv1D, MaxPool1D, Activation
from keras.models import Model

In [5]:
words = pd.read_table('dataset/words.txt', names = ['x'])
words = words.astype(str)

In [42]:
tok = Tokenizer()
tok.fit_on_texts(words['x'])
seq  = tok.texts_to_sequences(data)

v = len(tok.word_index)
slice_len = 80

In [61]:
max_len = max([len(i) for i in seq])
d = max_len / slice_len
if int(d) < d:
    max_len = int(d + 1) * slice_len
max_len = int(max_len)
padded = pad_sequences(seq, maxlen = max_len) #855 is divisible by 15
max_len

880

In [62]:
xx = padded / v

In [63]:
yy = to_categorical(y, num_classes=2)
11 * 80

880

In [64]:
yz = yy.reshape((6918, 1, 2))

In [66]:
xx.shape, yy.shape,
xz = xx.reshape((6918, int(max_len//slice_len), slice_len))

In [71]:
model = Sequential()  

model.add(Dense(input_shape = (None, slice_len), units = 1024))
model.add(Bidirectional(LSTM(input_shape = (None, slice_len), units = 512)))
model.add(Dense(units = 512, activation='tanh'))
model.add(Dense(units = 512, activation='sigmoid'))
model.add(Dense(2, activation='sigmoid'))
# model.add(Activation("sigmoid"))
model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
hists = []
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, None, 1024)        82944     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              6295552   
_________________________________________________________________
dense_32 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_33 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_34 (Dense)             (None, 2)                 1026      
Total params: 7,166,978
Trainable params: 7,166,978
Non-trainable params: 0
_________________________________________________________________


In [73]:
x_train = xz[:6000]
y_train = yy[:6000]
x_val = xz[6000:]
y_val = yy[6000:]


In [12]:
from IPython.display import clear_output


In [75]:
for i in range(10):
    hist = model.fit(x_train, y_train, 
                     batch_size=50, 
                     epochs = 10, 
                     verbose = 1,
                     shuffle = True,
                     validation_data = (x_val, y_val)
                    )
    hists.append(hist)
    clear_output()
    model.save_weights('model_56.hdf')

Train on 6000 samples, validate on 918 samples
Epoch 1/10
6000/6000 [==============================] - 147s - loss: 0.0311 - acc: 0.9612 - val_loss: 0.0588 - val_acc: 0.9259
Epoch 2/10
6000/6000 [==============================] - 147s - loss: 0.0297 - acc: 0.9620 - val_loss: 0.0594 - val_acc: 0.9281
Epoch 3/10
3650/6000 [=================>............] - ETA: 57s - loss: 0.0286 - acc: 0.9638

KeyboardInterrupt: 

In [76]:
import matplotlib.pyplot as plt

In [1]:
ax = plt.axis()
plt.axis(ax)
plt.figure(figsize=(500, 500))
# plt.suptitle("hello")
plt.plot(hist.history['acc'], label = 'accuracy')
plt.suptitle("Metrics")
plt.plot(hist.history['loss'], label = 'loss')
plt.legend()
plt.show()

NameError: name 'plt' is not defined

In [128]:
print(hist.history)

{'acc': [0.62483333001534147, 0.68133333325386047, 0.69566666980584457, 0.70749999384085338, 0.71350000103314715, 0.71683333416779837, 0.7230000009139379, 0.734333336353302, 0.74549999932448074, 0.74716666837533319], 'loss': [0.23150147919853528, 0.21405868083238602, 0.20481345901886622, 0.19799887736638386, 0.1945107673605283, 0.19240669583280881, 0.1871956095099449, 0.18118739848335583, 0.17555171959102153, 0.17373501683274906]}
